In [1]:
# Project Overview: Create Relevant Spotify Playlist with New Songs
# 0. Declare name of playlist to be created
# 1. Import Packages and establish creds to connect to Spotify
# 2. Get raw Spotify data of Liked (Hearted) songs into a dataframe
# 3. Add audio features
# 4. Add genre data
# 5. Create genre score
### Repeat process (steps 2-5) for disliked songs (based on dislike playlist) ###
# 6. Create playlist df (to access disliked playlist)
# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
# ....
# 10. Create playlist

In [9]:
# 0. Declare name of playlist to be created
pred_like_playlist_name = "GenreScore>20"
genre_threshold = 21

In [3]:
#### ALL FUNCTIONS BELOW ####

# 1. Import Packages and establish creds to connect to Spotify
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import requests
import math
import time
import pandas as pd
from IPython.display import display
from pandas.io.json import json_normalize
import numpy as np 
import matplotlib.pyplot as plt  
import inspect
import pickle

def define_scope():
    global username
    username = os.environ['SPOTIPY_USERNAME']
    global scope 
    scope = '''
    playlist-modify-private 
    playlist-modify-public 
    playlist-read-collaborative 
    playlist-read-private
    user-follow-modify 
    user-follow-read 
    user-library-modify 
    user-library-read 
    user-modify-playback-state 
    user-read-currently-playing 
    user-read-email 
    user-read-playback-state 
    user-read-private 
    user-read-recently-played 
    user-top-read'''
    global token, sp, user, headers
    try:
        token = util.prompt_for_user_token(username, scope)
    except (AttributeError, JSONDecodeError):
        os.remove(f".cache-{username}")
        token = util.prompt_for_user_token(username, scope)
    sp = spotipy.Spotify(auth=token)
    user = sp.current_user()
    headers = {'Authorization': "Bearer {}".format(token)}
    return

# 2. Get raw Spotify data of Liked (i.e. Hearted) songs into a dataframe
# 2a. Get raw data of liked songs into a list
def req(url,headers):
    r = requests.get(url, headers=headers)
    return json.loads(r.text)

def get_raw_liked_song_list():
    print('Get_raw_liked_song_list')
    offset = 0
    url = f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}"

    liked_song_count = req(url,headers=headers)["total"]
    print("Liked song count: ", liked_song_count)

    raw_liked_songs = []
    for i in range(int(math.ceil(liked_song_count/50.0))):
        offset = 50*i
        raw_liked_songs.extend(req(f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}",headers=headers)["items"])

    print(f">> raw liked songs['0'] \n>> {raw_liked_songs[0]}")
    return raw_liked_songs

# 2b. transform liked song raw data list into df
# parse out data from Liked Songs' raw data
def song_metadata_to_df(raw_songs):
    print('Transforming song raw data list into df\n')
    define_scope()
    # create empty df
    df = pd.DataFrame(columns = [
        'album', 
        'artist_uri', 
        'artist',
        'track',
        'url',
        'popularity',
        'added_at',
        'release_date',
        'release_year',
        'uri'
        ])
    # parse out songs 100 at a time
    for i in range(0, len(raw_songs)):
        if (i/100).is_integer():
            print(f"{i}/{len(raw_songs)} parsed")
        try:
            album = raw_songs[i]['track']['album']['name']
            artist_uri = raw_songs[i]['track']['artists'][0]['uri'].split(':')[2]
            artist = raw_songs[i]['track']['album']['artists'][0]['name']
            track = raw_songs[i]['track']['name']
            url = raw_songs[i]['track']['external_urls']['spotify']
            popularity = raw_songs[i]['track']['popularity']
            added_at = raw_songs[i]['added_at']
            release_date = raw_songs[i]['track']['album']['release_date']
            release_year = release_date.split('-')[0]
            uri = raw_songs[i]['track']['uri']
            df = df.append(pd.DataFrame([[
                album, 
                artist_uri,
                artist, 
                track, 
                url, 
                popularity, 
                added_at, 
                release_date, 
                release_year,
                uri
                ]], columns = [
                'album', 
                'artist_uri', 
                'artist',
                'track',
                'url',
                'popularity',
                'added_at',
                'release_date',
                'release_year',
                'uri'
                ]))
        except:
            pass
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

#3. Add audio features 
# Combine liked song df with audio feature df
def add_audio_feats(df):
    print('Combining song_df with audio_feature_df')
    # get list of song uris
    song_uri_list = df.index.to_list()
    audio_feature_df = pd.DataFrame()
    for i in range(0,math.ceil(len(song_uri_list)/100)):
        define_scope()
        features = sp.audio_features(song_uri_list[(i)*100:100*(i+1)])
        audio_feature_df = audio_feature_df.append(pd.DataFrame(features))
    audio_feature_df.set_index('uri', inplace=True)
    # merge this 
    liked_song_df = df.merge(audio_feature_df, on = 'uri')
    display(liked_song_df.head())
    return liked_song_df.drop_duplicates()

#4. Add genre data
# 4a. Get artist_uris -> get genres
def get_genres(df):
    print('Getting genres\n')
    g = []
    artist_uris = df['artist_uri'].unique()
    print(len(artist_uris), ' distinct artists')
    for i in range(0, len(artist_uris)):
        if (i/100).is_integer():
            print(f"{i}/{len(artist_uris)} parsed")
        try:
            g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
        except:
            define_scope()
            try:
                g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
            except:
                pass
    # genres_df's artist_uri column stores a list of genres per artist uri
    genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
    
    print('\ngenre_df')
    display(genres_df.head())
    return genres_df

# There is a list of genres for one artist that must be broken out
def explode_genres(df):
    genre_exploded_df = df.explode('genre list')
    
    print('\ngenre_exploded_df')
    display(genre_exploded_df.head())
    return genre_exploded_df

# Add these genres back to the df for each song
def add_genres(df, genre_exploded_df):
    # the following does not work oddly enough
    # liked_song_df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri', inplace = True)
    df = df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri')
    df.rename(columns={'genre list': 'genre'}, inplace=True)
    
    print('Merging genres into song_df')
    display(df.head())
    return df

#5. Create Genre Score

# Get genre counts for liked songs - the higher the count, the higher the score
def get_genre_counts(genre_exploded_df):
    print('Getting genre counts')
    # establish genre counts (which will determine score)
    genre_count_df = pd.DataFrame(genre_exploded_df['genre list'].value_counts()).reset_index()
    genre_count_df.columns = ['genre','genre count']
    print('\ngenre_count_df.head()')
    display(genre_count_df.head())
    return genre_count_df

# Merge the genre counts with the song df and sum the genre column
def establish_genre_score(genre_count_df, df):
    print('\nMerging genre counts with song_df')
    df = df.reset_index().merge(genre_count_df , on = 'genre',how = 'left')
    df.drop(columns=['genre'], inplace=True)
    display(df.head())

    groupby_cols = df.columns.to_list()[:-1]
    
    print('\nSumming genre column')
    df = df.groupby(groupby_cols).sum().sort_values('genre count', ascending=False)
    df = df.rename({'genre count': 'genre score'}, axis=1).reset_index() 
    # reset index after summing - otherwise everything prior to sum will become index
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

# 6. Create playlist df (to access disliked playlist)
# get playlist raw data (for dislikes)
def get_playlist_uris(playlists_raw):
    df = pd.DataFrame()
    for i in range(0, len(playlists_raw)):
        try:
            playlist = playlists_raw[i]['name']
        except:
            playlist = ''
        try:
            playlist_uri = playlists_raw[i]['uri']
        except:
            playlist_uri = ''
        try:
            song_count = playlists_raw[i]['tracks']['total']
        except:
            song_count = 0
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    print('playlist_df')
    display(df)
    return df

def get_raw_disliked_song_list():
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    display(playlists_df)
    uri = playlists_df.loc['Dislikes']['playlist_uri']
    song_count = playlists_df.loc['Dislikes']['song_count']
    # extract data from Disliked songs
    dislike_raw_list = []
    for i in range(int(math.ceil(song_count/100.0))):
        dislike_raw_list.extend(sp.user_playlist_tracks(uri.split(':')[1], uri.split(':')[2], offset=i*100)['items'])
    print('Getting raw disliked song list')
    return dislike_raw_list

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
def combine_dfs_to_create_my_songs_df(df1,df2, save_destination):
    print('Combining dfs to create my_songs_df and storing as pickle file')
    df1['Liked'] = 1
    df2['Liked'] = 0
    my_songs_df = pd.concat([df1,df2])
    my_songs_df.to_csv(f"{save_destination}/my_songs.csv")
    my_songs_df.to_pickle("./my_songs.pkl")
    display(my_songs_df.head())
    print('successfully saved to .csv and .pkl files in save_destination')

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
def get_featured_playlist_uris():
    print('Getting raw_featured_playlist_df')
    url = f"https://api.spotify.com/v1/browse/featured-playlists?limit=50&offset=0"
    featured_playlist_count = req(url,headers=headers)['playlists']['total']
    df = pd.DataFrame()
    for i in range(0,featured_playlist_count):
        raw_featured_playlist = req(url,headers=headers)
        playlist = raw_featured_playlist['playlists']['items'][i]['name']
        playlist_uri = raw_featured_playlist['playlists']['items'][i]['uri']
        song_count = raw_featured_playlist['playlists']['items'][i]['tracks']['total']
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
def get_raw_featured_playlist_song_list():
    playlists_df = get_featured_playlist_uris()
    raw_featured_songs = []
    for uri in playlists_df.loc[:,'playlist_uri'].to_list():
        song_count = playlists_df.loc[playlists_df['playlist_uri']==uri,'song_count'][0]
        offset = 0
        url = f"https://api.spotify.com/v1/playlists/{uri.split(':')[2]}/tracks?limit=50&offset={offset}"
        for i in range(int(math.ceil(song_count/100.0))):
            offset = 100*i
            raw_featured_songs.extend(req(url,headers=headers)['items'])

    return raw_featured_songs

def create_playlist(pred_like_playlist_name, df, genre_score_threshold):
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    playlist_uri = get_playlist_uris(playlists_raw).loc[playlists_df.index == pred_like_playlist_name,'playlist_uri'][0].split(':')[2]
    uri_list = df.loc[df['genre score']>= genre_score_threshold].reset_index()['uri'].to_list()
    num_songs_to_add = len(uri_list)
    for i in range(int(math.ceil(num_songs_to_add/100.0))):
        url = f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?uris={','.join(uri_list[i*100:(i+1)*100])}"
        requests.post(url, headers=headers)
    return

In [4]:
define_scope()
print('Scope defined\n****PART 1: Get Liked Songs Dataframe****\n')
raw_liked_song_list = get_raw_liked_song_list()
liked_song_df = song_metadata_to_df(raw_liked_song_list)
liked_song_df = add_audio_feats(liked_song_df)
genre_df = get_genres(liked_song_df)
genre_exploded_df = explode_genres(genre_df)
liked_song_df = add_genres(liked_song_df, genre_exploded_df)
liked_genre_count_df = get_genre_counts(genre_exploded_df)
liked_song_df = establish_genre_score(liked_genre_count_df, liked_song_df)

Scope defined
****PART 1: Get Liked Songs Dataframe****

Get_raw_liked_song_list
Liked song count:  616
>> raw liked songs['0'] 
>> {'added_at': '2020-09-06T20:15:50Z', 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/46njgd2Rq9tZc4ZjeQMgbh'}, 'href': 'https://api.spotify.com/v1/artists/46njgd2Rq9tZc4ZjeQMgbh', 'id': '46njgd2Rq9tZc4ZjeQMgbh', 'name': 'Boz Scaggs', 'type': 'artist', 'uri': 'spotify:artist:46njgd2Rq9tZc4ZjeQMgbh'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:0nlZjTPzW6S9JIESuGfImT,Silk Degrees,46njgd2Rq9tZc4ZjeQMgbh,Boz Scaggs,Lowdown,https://open.spotify.com/track/0nlZjTPzW6S9JIE...,63,2020-09-06T20:15:50Z,1976,1976
spotify:track:0W5qBuHbbBfCzszFxrQWMc,Half the City,4fXkvh05wFhuH77MfD4m9o,St. Paul & The Broken Bones,Call Me,https://open.spotify.com/track/0W5qBuHbbBfCzsz...,61,2020-09-06T17:40:18Z,2014-02-18,2014
spotify:track:5cNfWI5CJs7ADjSC9SWRPX,The Dustbowl Symphony,5CAiy5rDr3LaSoGtJOq6A0,Nanci Griffith,Love at the Five and Dime,https://open.spotify.com/track/5cNfWI5CJs7ADjS...,14,2020-09-06T04:19:37Z,1999-09-03,1999
spotify:track:7b6fpjQcQcGDfpG7hA3P2k,Austin City Limits Festival,38euc5ce2q2MMzaoJcPstz,Various Artists,It's The Nighttime,https://open.spotify.com/track/7b6fpjQcQcGDfpG...,6,2020-09-06T04:19:36Z,2005-08-30,2005
spotify:track:10e1Jjuo9NviGDjpvJGKIJ,Kamaal The Abstract,3ZotbHeyVQKxQCPDJuQ4SU,Q-Tip,Barely In Love,https://open.spotify.com/track/10e1Jjuo9NviGDj...,40,2020-08-31T22:33:48Z,2009-09-15,2009


Combining song_df with audio_feature_df


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:0nlZjTPzW6S9JIESuGfImT,Silk Degrees,46njgd2Rq9tZc4ZjeQMgbh,Boz Scaggs,Lowdown,https://open.spotify.com/track/0nlZjTPzW6S9JIE...,63,2020-09-06T20:15:50Z,1976,1976,0.662,...,0.141000,0.0351,0.929,116.499,audio_features,0nlZjTPzW6S9JIESuGfImT,https://api.spotify.com/v1/tracks/0nlZjTPzW6S9...,https://api.spotify.com/v1/audio-analysis/0nlZ...,315640,4
spotify:track:0W5qBuHbbBfCzszFxrQWMc,Half the City,4fXkvh05wFhuH77MfD4m9o,St. Paul & The Broken Bones,Call Me,https://open.spotify.com/track/0W5qBuHbbBfCzsz...,61,2020-09-06T17:40:18Z,2014-02-18,2014,0.573,...,0.211000,0.1170,0.610,110.289,audio_features,0W5qBuHbbBfCzszFxrQWMc,https://api.spotify.com/v1/tracks/0W5qBuHbbBfC...,https://api.spotify.com/v1/audio-analysis/0W5q...,171267,4
spotify:track:5cNfWI5CJs7ADjSC9SWRPX,The Dustbowl Symphony,5CAiy5rDr3LaSoGtJOq6A0,Nanci Griffith,Love at the Five and Dime,https://open.spotify.com/track/5cNfWI5CJs7ADjS...,14,2020-09-06T04:19:37Z,1999-09-03,1999,0.335,...,0.000000,0.2410,0.375,67.626,audio_features,5cNfWI5CJs7ADjSC9SWRPX,https://api.spotify.com/v1/tracks/5cNfWI5CJs7A...,https://api.spotify.com/v1/audio-analysis/5cNf...,294667,4
spotify:track:7b6fpjQcQcGDfpG7hA3P2k,Austin City Limits Festival,38euc5ce2q2MMzaoJcPstz,Various Artists,It's The Nighttime,https://open.spotify.com/track/7b6fpjQcQcGDfpG...,6,2020-09-06T04:19:36Z,2005-08-30,2005,0.574,...,0.000013,0.9320,0.695,131.563,audio_features,7b6fpjQcQcGDfpG7hA3P2k,https://api.spotify.com/v1/tracks/7b6fpjQcQcGD...,https://api.spotify.com/v1/audio-analysis/7b6f...,243520,4
spotify:track:10e1Jjuo9NviGDjpvJGKIJ,Kamaal The Abstract,3ZotbHeyVQKxQCPDJuQ4SU,Q-Tip,Barely In Love,https://open.spotify.com/track/10e1Jjuo9NviGDj...,40,2020-08-31T22:33:48Z,2009-09-15,2009,0.932,...,0.020900,0.1490,0.697,120.018,audio_features,10e1Jjuo9NviGDjpvJGKIJ,https://api.spotify.com/v1/tracks/10e1Jjuo9Nvi...,https://api.spotify.com/v1/audio-analysis/10e1...,243307,1


Getting genres

394  distinct artists
0/394 parsed
100/394 parsed
200/394 parsed
300/394 parsed

genre_df


...

,genre,genre count
0,indie pop,75
1,indie rock,72
2,stomp and holler,65
3,indie folk,60
4,modern rock,51



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:0nlZjTPzW6S9JIESuGfImT,Silk Degrees,46njgd2Rq9tZc4ZjeQMgbh,Boz Scaggs,Lowdown,https://open.spotify.com/track/0nlZjTPzW6S9JIE...,63,2020-09-06T20:15:50Z,1976,1976,...,0.0351,0.929,116.499,audio_features,0nlZjTPzW6S9JIESuGfImT,https://api.spotify.com/v1/tracks/0nlZjTPzW6S9...,https://api.spotify.com/v1/audio-analysis/0nlZ...,315640,4,2.0
1,spotify:track:0nlZjTPzW6S9JIESuGfImT,Silk Degrees,46njgd2Rq9tZc4ZjeQMgbh,Boz Scaggs,Lowdown,https://open.spotify.com/track/0nlZjTPzW6S9JIE...,63,2020-09-06T20:15:50Z,1976,1976,...,0.0351,0.929,116.499,audio_features,0nlZjTPzW6S9JIESuGfImT,https://api.spotify.com/v1/tracks/0nlZjTPzW6S9...,https://api.spotify.com/v1/audio-analysis/0nlZ...,315640,4,11.0
2,spotify:track:0nlZjTPzW6S9JIESuGfImT,Silk Degrees,46njgd2Rq9tZc4ZjeQMgbh,Boz Scaggs,Lowdown,https://open.spotify.com/track/0nlZjTPzW6S9JIE...,63,2020-09-06T20:15:50Z,1976,1976,...,0.0351,0.929,116.499,audio_features,0nlZjTPzW6S9JIESuGfImT,https://api.spotify.com/v1/tracks/0nlZjTPzW6S9...,https://api.spotify.com/v1/audio-analysis/0nlZ...,315640,4,13.0
3,spotify:track:0nlZjTPzW6S9JIESuGfImT,Silk Degrees,46njgd2Rq9tZc4ZjeQMgbh,Boz Scaggs,Lowdown,https://open.spotify.com/track/0nlZjTPzW6S9JIE...,63,2020-09-06T20:15:50Z,1976,1976,...,0.0351,0.929,116.499,audio_features,0nlZjTPzW6S9JIESuGfImT,https://api.spotify.com/v1/tracks/0nlZjTPzW6S9...,https://api.spotify.com/v1/audio-analysis/0nlZ...,315640,4,14.0
4,spotify:track:0nlZjTPzW6S9JIESuGfImT,Silk Degrees,46njgd2Rq9tZc4ZjeQMgbh,Boz Scaggs,Lowdown,https://open.spotify.com/track/0nlZjTPzW6S9JIE...,63,2020-09-06T20:15:50Z,1976,1976,...,0.0351,0.929,116.499,audio_features,0nlZjTPzW6S9JIESuGfImT,https://api.spotify.com/v1/tracks/0nlZjTPzW6S9...,https://api.spotify.com/v1/audio-analysis/0nlZ...,315640,4,11.0



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,47,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.0979,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,484.0
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,53,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.0961,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,484.0
spotify:track:7ywp9MWERblgkSMoMBugC0,Holiday,6RWjTQqILL7a1tQ0VapyLK,The Magnetic Fields,Strange Powers,https://open.spotify.com/track/7ywp9MWERblgkSM...,51,2019-11-03T17:42:52Z,1994,1994,0.668,...,0.1290,0.963,131.767,audio_features,7ywp9MWERblgkSMoMBugC0,https://api.spotify.com/v1/tracks/7ywp9MWERblg...,https://api.spotify.com/v1/audio-analysis/7ywp...,161613,4,447.0
spotify:track:0iTpQYzJnYgh7kIxyq8A2O,Veckatimest,2Jv5eshHtLycR6R8KQCdc4,Grizzly Bear,Two Weeks,https://open.spotify.com/track/0iTpQYzJnYgh7kI...,70,2019-11-03T17:35:40Z,2009-05-24,2009,0.537,...,0.0800,0.704,159.871,audio_features,0iTpQYzJnYgh7kIxyq8A2O,https://api.spotify.com/v1/tracks/0iTpQYzJnYgh...,https://api.spotify.com/v1/audio-analysis/0iTp...,243160,4,441.0
spotify:track:0szSOfZVM3G5mWI1ogQjr7,Painted Ruins,2Jv5eshHtLycR6R8KQCdc4,Grizzly Bear,Mourning Sound,https://open.spotify.com/track/0szSOfZVM3G5mWI...,54,2019-11-03T17:41:44Z,2017-08-18,2017,0.507,...,0.1220,0.441,128.015,audio_features,0szSOfZVM3G5mWI1ogQjr7,https://api.spotify.com/v1/tracks/0szSOfZVM3G5...,https://api.spotify.com/v1/audio-analysis/0szS...,262107,4,441.0


In [5]:
print('\n****PART 2: Get Disliked Songs Dataframe****\n')
define_scope()
raw_disliked_song_list = get_raw_disliked_song_list()
disliked_song_df = song_metadata_to_df(raw_disliked_song_list)
disliked_song_df = add_audio_feats(disliked_song_df)
genre_df = get_genres(disliked_song_df)
genre_exploded_df = explode_genres(genre_df)
disliked_song_df = add_genres(disliked_song_df, genre_exploded_df)
disliked_genre_count_df = get_genre_counts(genre_exploded_df)
disliked_genre_count_df['genre count'] = disliked_genre_count_df['genre count'] * -1
genre_count_df = pd.concat([liked_genre_count_df, disliked_genre_count_df])
disliked_song_df = establish_genre_score(disliked_genre_count_df, disliked_song_df)


****PART 2: Get Disliked Songs Dataframe****

playlist_df


,playlist_uri,song_count
playlist,,
GenreScoreGreaterThan0,spotify:playlist:3KaraMEES4TUofPK4qse7j,436
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,448
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,playlist_uri,song_count
playlist,,
GenreScoreGreaterThan0,spotify:playlist:3KaraMEES4TUofPK4qse7j,436
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,448
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


Getting raw disliked song list
Transforming song raw data list into df

0/448 parsed
100/448 parsed
200/448 parsed
300/448 parsed
400/448 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,40,2019-11-06T02:39:29Z,2012-01-01,2012
spotify:track:6muOWhMh7Tf0UFUtMDBhUR,Orphans / Arabesque,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,Orphans,https://open.spotify.com/track/6muOWhMh7Tf0UFU...,69,2019-11-06T02:40:45Z,2019-10-24,2019
spotify:track:5jafMI8FLibnjkYTZ33m0c,The Bends,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,High And Dry,https://open.spotify.com/track/5jafMI8FLibnjkY...,74,2019-11-06T02:45:04Z,1995-03-28,1995
spotify:track:04KJ4NGb2T22y26sTO41Q0,Rise Of An Empire (Deluxe Edition),5OrB6Jhhrl9y2PK0pSV4VP,Young Money,Moment,https://open.spotify.com/track/04KJ4NGb2T22y26...,40,2019-11-06T02:59:28Z,2014-01-01,2014
spotify:track:6sICylTFKeiplGHltWnJ1k,Z,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,What A Wonderful Man,https://open.spotify.com/track/6sICylTFKeiplGH...,35,2019-11-06T03:02:29Z,2005-10-04,2005


...

,genre,genre count
0,dance pop,44
1,pop,37
2,pop rap,33
3,rock,32
4,alternative rock,30



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,40,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-44.0
1,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,40,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-21.0
2,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,40,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-37.0
3,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,40,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-33.0
4,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,40,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-16.0



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:1TKRumlRMXv2SEKPhueya0,The Step Daddy (Explicit),6RhzdrHfuZEMnZTNRjbTpz,Hitman Sammy Sam,Step Daddy,https://open.spotify.com/track/1TKRumlRMXv2SEK...,26,2020-04-02T13:23:41Z,2003-01-01,2003,0.930,...,0.0767,0.348,118.997,audio_features,1TKRumlRMXv2SEKPhueya0,https://api.spotify.com/v1/tracks/1TKRumlRMXv2...,https://api.spotify.com/v1/audio-analysis/1TKR...,230493,4,0.0
spotify:track:6QeMRyd3BmSfJbaNay3dpe,R&B - 100 Hits - The Greatest R n B album - 10...,2mxe0TnaNL039ysAj51xPQ,Various Artists,The World's Greatest,https://open.spotify.com/track/6QeMRyd3BmSfJba...,54,2020-04-25T23:19:22Z,2013-06-21,2013,0.661,...,0.0469,0.721,95.971,audio_features,6QeMRyd3BmSfJbaNay3dpe,https://api.spotify.com/v1/tracks/6QeMRyd3BmSf...,https://api.spotify.com/v1/audio-analysis/6QeM...,277600,4,0.0
spotify:track:5nB5bek72WORC5jegCIPNk,Untitled,2mxe0TnaNL039ysAj51xPQ,R. Kelly,Number One,https://open.spotify.com/track/5nB5bek72WORC5j...,41,2020-04-02T16:30:51Z,2009-11-27,2009,0.650,...,0.1800,0.576,136.796,audio_features,5nB5bek72WORC5jegCIPNk,https://api.spotify.com/v1/tracks/5nB5bek72WOR...,https://api.spotify.com/v1/audio-analysis/5nB5...,260867,4,0.0
spotify:track:00nQhTtc8FxgmB0NWypSE0,In Good Company,62W2NNKgzSx7GFJDUbJD6L,Java Gospel,You Rock My World,https://open.spotify.com/track/00nQhTtc8FxgmB0...,0,2020-04-02T13:32:30Z,2007-07-11,2007,0.662,...,0.3330,0.915,121.961,audio_features,00nQhTtc8FxgmB0NWypSE0,https://api.spotify.com/v1/tracks/00nQhTtc8Fxg...,https://api.spotify.com/v1/audio-analysis/00nQ...,245947,4,0.0
spotify:track:2x5C1qAOJ1KJFqM6uhOsgT,Blocked Numbers,2GdgDnUG0ZMFZiu2122pGQ,Crystal Skulls,Hussy,https://open.spotify.com/track/2x5C1qAOJ1KJFqM...,17,2020-06-29T13:48:04Z,2005-01-01,2005,0.660,...,0.1290,0.792,119.568,audio_features,2x5C1qAOJ1KJFqM6uhOsgT,https://api.spotify.com/v1/tracks/2x5C1qAOJ1KJ...,https://api.spotify.com/v1/audio-analysis/2x5C...,279093,4,0.0


In [6]:
print('\n****PART 3: Combine liked and disliked songs into my_songs dataframe with "Liked" flag and saving as .csv and pickle file***\n')
my_songs_df = combine_dfs_to_create_my_songs_df(liked_song_df,disliked_song_df,'/Users/Will/Documents/Spotipy')
#my_songs_df = pd.read_pickle("./my_songs.pkl")


****PART 3: Combine liked and disliked songs into my_songs dataframe with "Liked" flag and saving as .csv and pickle file***

Combining dfs to create my_songs_df and storing as pickle file


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score,Liked
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,47,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,484.0,1
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,53,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,484.0,1
spotify:track:7ywp9MWERblgkSMoMBugC0,Holiday,6RWjTQqILL7a1tQ0VapyLK,The Magnetic Fields,Strange Powers,https://open.spotify.com/track/7ywp9MWERblgkSM...,51,2019-11-03T17:42:52Z,1994,1994,0.668,...,0.963,131.767,audio_features,7ywp9MWERblgkSMoMBugC0,https://api.spotify.com/v1/tracks/7ywp9MWERblg...,https://api.spotify.com/v1/audio-analysis/7ywp...,161613,4,447.0,1
spotify:track:0iTpQYzJnYgh7kIxyq8A2O,Veckatimest,2Jv5eshHtLycR6R8KQCdc4,Grizzly Bear,Two Weeks,https://open.spotify.com/track/0iTpQYzJnYgh7kI...,70,2019-11-03T17:35:40Z,2009-05-24,2009,0.537,...,0.704,159.871,audio_features,0iTpQYzJnYgh7kIxyq8A2O,https://api.spotify.com/v1/tracks/0iTpQYzJnYgh...,https://api.spotify.com/v1/audio-analysis/0iTp...,243160,4,441.0,1
spotify:track:0szSOfZVM3G5mWI1ogQjr7,Painted Ruins,2Jv5eshHtLycR6R8KQCdc4,Grizzly Bear,Mourning Sound,https://open.spotify.com/track/0szSOfZVM3G5mWI...,54,2019-11-03T17:41:44Z,2017-08-18,2017,0.507,...,0.441,128.015,audio_features,0szSOfZVM3G5mWI1ogQjr7,https://api.spotify.com/v1/tracks/0szSOfZVM3G5...,https://api.spotify.com/v1/audio-analysis/0szS...,262107,4,441.0,1


successfully saved to .csv and .pkl files in save_destination


In [7]:
print('\n****PART 4: Get songs from featured playlists and score them based on genre****\n')
raw_featured_playlist_songs_list = get_raw_featured_playlist_song_list()
featured_playlist_song_df = song_metadata_to_df(raw_featured_playlist_songs_list)
featured_playlist_song_df = add_audio_feats(featured_playlist_song_df)
genre_df = get_genres(featured_playlist_song_df)
genre_exploded_df = explode_genres(genre_df)
featured_playlist_song_df = add_genres(featured_playlist_song_df, genre_exploded_df)
featured_playlist_song_df = establish_genre_score(genre_count_df, featured_playlist_song_df)


****PART 4: Get songs from featured playlists and score them based on genre****

Getting raw_featured_playlist_df


,playlist_uri,song_count
playlist,,
Calm Vibes,spotify:playlist:37i9dQZF1DX1s9knjP51Oa,219
Sleep,spotify:playlist:37i9dQZF1DWZd79rJ6a7lp,270
Bliss,spotify:playlist:37i9dQZF1DX4bSrsRWE9cd,74
Soft Pop Hits,spotify:playlist:37i9dQZF1DWTwnEm1IYyoj,234
Rain Sounds,spotify:playlist:37i9dQZF1DX8ymr6UES7vc,99
Night Rain,spotify:playlist:37i9dQZF1DXbcPC6Vvqudd,123
Late Night Jazz,spotify:playlist:37i9dQZF1DX4wta20PHgwo,70
ASMR Sleep,spotify:playlist:37i9dQZF1DWUAeTOoyNaqm,51
Relax & Unwind,spotify:playlist:37i9dQZF1DWU0ScTcjJBdj,100


Transforming song raw data list into df

0/1050 parsed
100/1050 parsed
200/1050 parsed
300/1050 parsed
400/1050 parsed
500/1050 parsed
600/1050 parsed
700/1050 parsed
800/1050 parsed
900/1050 parsed
1000/1050 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:0r8M4zqlJeV0XPtCxyQ6fn,Auriga,5El7zD8iCQu87fDwPBlPbN,Odin Sørlie,Auriga,https://open.spotify.com/track/0r8M4zqlJeV0XPt...,61,2020-09-01T06:55:59Z,2020-05-29,2020
spotify:track:4YxLJVcYXgN1Kxb67BvxEZ,Softly Spoken,5lvoIhUaKTZ5DPNElfgAu6,Atticus Laines,Softly Spoken,https://open.spotify.com/track/4YxLJVcYXgN1Kxb...,57,2020-09-01T06:55:59Z,2020-08-28,2020
spotify:track:6gDt7eZjJNIkzLj9jVEBZo,First Sunrise,6qjQ8iKIjHkc6x5wodNJiD,Haru Kitamura,First Sunrise,https://open.spotify.com/track/6gDt7eZjJNIkzLj...,65,2020-09-01T06:55:59Z,2020-02-12,2020
spotify:track:6I2mlN0j24YN87ke0mAthD,Ti Penso,0z1HXjcSk4VetGgiP5qD5n,Gianina Cremonesi,Ti Penso,https://open.spotify.com/track/6I2mlN0j24YN87k...,57,2020-09-01T06:55:59Z,2020-08-07,2020
spotify:track:4XnsaU3RkUS7o0nqLIUqW7,Beneath Your Smile,3eV4rEEDa28eEDqtKAz99w,James DiResta,Beneath Your Smile,https://open.spotify.com/track/4XnsaU3RkUS7o0n...,63,2020-09-01T06:55:59Z,2020-08-21,2020


Combining song_df with audio_feature_df


...

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:0r8M4zqlJeV0XPtCxyQ6fn,Auriga,5El7zD8iCQu87fDwPBlPbN,Odin Sørlie,Auriga,https://open.spotify.com/track/0r8M4zqlJeV0XPt...,61,2020-09-01T06:55:59Z,2020-05-29,2020,0.398,...,0.130,0.612,72.166,audio_features,0r8M4zqlJeV0XPtCxyQ6fn,https://api.spotify.com/v1/tracks/0r8M4zqlJeV0...,https://api.spotify.com/v1/audio-analysis/0r8M...,134500,4,background piano
spotify:track:4YxLJVcYXgN1Kxb67BvxEZ,Softly Spoken,5lvoIhUaKTZ5DPNElfgAu6,Atticus Laines,Softly Spoken,https://open.spotify.com/track/4YxLJVcYXgN1Kxb...,57,2020-09-01T06:55:59Z,2020-08-28,2020,0.527,...,0.120,0.736,69.758,audio_features,4YxLJVcYXgN1Kxb67BvxEZ,https://api.spotify.com/v1/tracks/4YxLJVcYXgN1...,https://api.spotify.com/v1/audio-analysis/4YxL...,136692,3,NaN
spotify:track:6gDt7eZjJNIkzLj9jVEBZo,First Sunrise,6qjQ8iKIjHkc6x5wodNJiD,Haru Kitamura,First Sunrise,https://open.spotify.com/track/6gDt7eZjJNIkzLj...,65,2020-09-01T06:55:59Z,2020-02-12,2020,0.483,...,0.102,0.108,94.135,audio_features,6gDt7eZjJNIkzLj9jVEBZo,https://api.spotify.com/v1/tracks/6gDt7eZjJNIk...,https://api.spotify.com/v1/audio-analysis/6gDt...,212000,4,background piano
spotify:track:6I2mlN0j24YN87ke0mAthD,Ti Penso,0z1HXjcSk4VetGgiP5qD5n,Gianina Cremonesi,Ti Penso,https://open.spotify.com/track/6I2mlN0j24YN87k...,57,2020-09-01T06:55:59Z,2020-08-07,2020,0.637,...,0.102,0.309,64.998,audio_features,6I2mlN0j24YN87ke0mAthD,https://api.spotify.com/v1/tracks/6I2mlN0j24YN...,https://api.spotify.com/v1/audio-analysis/6I2m...,121831,3,NaN
spotify:track:4XnsaU3RkUS7o0nqLIUqW7,Beneath Your Smile,3eV4rEEDa28eEDqtKAz99w,James DiResta,Beneath Your Smile,https://open.spotify.com/track/4XnsaU3RkUS7o0n...,63,2020-09-01T06:55:59Z,2020-08-21,2020,0.367,...,0.108,0.224,116.969,audio_features,4XnsaU3RkUS7o0nqLIUqW7,https://api.spotify.com/v1/tracks/4XnsaU3RkUS7...,https://api.spotify.com/v1/audio-analysis/4Xns...,286250,4,NaN



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:0r8M4zqlJeV0XPtCxyQ6fn,Auriga,5El7zD8iCQu87fDwPBlPbN,Odin Sørlie,Auriga,https://open.spotify.com/track/0r8M4zqlJeV0XPt...,61,2020-09-01T06:55:59Z,2020-05-29,2020,...,0.130,0.612,72.166,audio_features,0r8M4zqlJeV0XPtCxyQ6fn,https://api.spotify.com/v1/tracks/0r8M4zqlJeV0...,https://api.spotify.com/v1/audio-analysis/0r8M...,134500,4,-1.0
1,spotify:track:4YxLJVcYXgN1Kxb67BvxEZ,Softly Spoken,5lvoIhUaKTZ5DPNElfgAu6,Atticus Laines,Softly Spoken,https://open.spotify.com/track/4YxLJVcYXgN1Kxb...,57,2020-09-01T06:55:59Z,2020-08-28,2020,...,0.120,0.736,69.758,audio_features,4YxLJVcYXgN1Kxb67BvxEZ,https://api.spotify.com/v1/tracks/4YxLJVcYXgN1...,https://api.spotify.com/v1/audio-analysis/4YxL...,136692,3,NaN
2,spotify:track:6gDt7eZjJNIkzLj9jVEBZo,First Sunrise,6qjQ8iKIjHkc6x5wodNJiD,Haru Kitamura,First Sunrise,https://open.spotify.com/track/6gDt7eZjJNIkzLj...,65,2020-09-01T06:55:59Z,2020-02-12,2020,...,0.102,0.108,94.135,audio_features,6gDt7eZjJNIkzLj9jVEBZo,https://api.spotify.com/v1/tracks/6gDt7eZjJNIk...,https://api.spotify.com/v1/audio-analysis/6gDt...,212000,4,-1.0
3,spotify:track:6I2mlN0j24YN87ke0mAthD,Ti Penso,0z1HXjcSk4VetGgiP5qD5n,Gianina Cremonesi,Ti Penso,https://open.spotify.com/track/6I2mlN0j24YN87k...,57,2020-09-01T06:55:59Z,2020-08-07,2020,...,0.102,0.309,64.998,audio_features,6I2mlN0j24YN87ke0mAthD,https://api.spotify.com/v1/tracks/6I2mlN0j24YN...,https://api.spotify.com/v1/audio-analysis/6I2m...,121831,3,NaN
4,spotify:track:4XnsaU3RkUS7o0nqLIUqW7,Beneath Your Smile,3eV4rEEDa28eEDqtKAz99w,James DiResta,Beneath Your Smile,https://open.spotify.com/track/4XnsaU3RkUS7o0n...,63,2020-09-01T06:55:59Z,2020-08-21,2020,...,0.108,0.224,116.969,audio_features,4XnsaU3RkUS7o0nqLIUqW7,https://api.spotify.com/v1/tracks/4XnsaU3RkUS7...,https://api.spotify.com/v1/audio-analysis/4Xns...,286250,4,NaN



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:7d0XkI4iJ32nxeFAHFXbLQ,Crack-Up,4EVpmkEwrLYEg6jIsiPMIb,Fleet Foxes,"If You Need To, Keep Time on Me",https://open.spotify.com/track/7d0XkI4iJ32nxeF...,65,2020-05-18T11:00:36Z,2017-06-16,2017,0.521,...,0.179,0.135,104.030,audio_features,7d0XkI4iJ32nxeFAHFXbLQ,https://api.spotify.com/v1/tracks/7d0XkI4iJ32n...,https://api.spotify.com/v1/audio-analysis/7d0X...,210507,4,252.0
spotify:track:2Iwx2DFtAkA7M29ndlRtKv,Golden Age Of Radio,6igfLpd8s6DBBAuwebRUuo,Josh Ritter,Come and Find Me,https://open.spotify.com/track/2Iwx2DFtAkA7M29...,12,2020-05-18T11:00:36Z,2005-01-01,2005,0.414,...,0.101,0.480,92.425,audio_features,2Iwx2DFtAkA7M29ndlRtKv,https://api.spotify.com/v1/tracks/2Iwx2DFtAkA7...,https://api.spotify.com/v1/audio-analysis/2Iwx...,229427,4,249.0
spotify:track:3HQdwlovzULm18PjvlA5wz,Eyes,2JSc53B5cQ31m0xTB7JFpG,Rogue Wave,Eyes,https://open.spotify.com/track/3HQdwlovzULm18P...,58,2020-05-18T11:00:36Z,2006-02-12,2006,0.642,...,0.106,0.484,130.018,audio_features,3HQdwlovzULm18PjvlA5wz,https://api.spotify.com/v1/tracks/3HQdwlovzULm...,https://api.spotify.com/v1/audio-analysis/3HQd...,148867,4,225.0
spotify:track:1NCAKlp1bFHYbSSDQxfbJ1,Sleeping Operator,4OyRutd80DZC22C4pl63l7,The Barr Brothers,Even The Darkness Has Arms,https://open.spotify.com/track/1NCAKlp1bFHYbSS...,0,2020-09-04T20:50:51Z,2014-10-07,2014,0.513,...,0.288,0.734,101.275,audio_features,1NCAKlp1bFHYbSSDQxfbJ1,https://api.spotify.com/v1/tracks/1NCAKlp1bFHY...,https://api.spotify.com/v1/audio-analysis/1NCA...,230520,4,209.0
spotify:track:7kqnoZSZUifYDyR5jzJN6v,Spotify Singles,69tiO1fG8VWduDl3ji2qhI,Mt. Joy,Silver Lining - Recorded at Spotify Studios NYC,https://open.spotify.com/track/7kqnoZSZUifYDyR...,52,2020-09-04T20:50:51Z,2018-07-18,2018,0.488,...,0.166,0.338,138.893,audio_features,7kqnoZSZUifYDyR5jzJN6v,https://api.spotify.com/v1/tracks/7kqnoZSZUifY...,https://api.spotify.com/v1/audio-analysis/7kqn...,209027,4,196.0


In [10]:
create_playlist(pred_like_playlist_name, featured_playlist_song_df, genre_threshold)

playlist_df


,playlist_uri,song_count
playlist,,
GenreScore>0,spotify:playlist:33sE9iOLBfg3am9mtSk2k7,0
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,448
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


playlist_df


,playlist_uri,song_count
playlist,,
GenreScore>0,spotify:playlist:33sE9iOLBfg3am9mtSk2k7,0
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,448
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30
